In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Cargar desde pickle (lo que ya guardamos antes)
X = pd.read_pickle("X_processed.pkl")
y = pd.read_pickle("y_labels.pkl")["Credit_Score"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


((80000, 6700), (20000, 6700))

Escalar los datos con StandardScaler

PCA exige escalado, así que primero normalizamos X_train y X_test.

In [2]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# OJO: fit SOLO con train
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

X_train_scaled.shape, X_test_scaled.shape


((80000, 6700), (20000, 6700))

Definir el modelo elegido (Random Forest)

Usamos el mismo tipo de modelo que ya elegiste como mejor (RandomForest).

Puedes fijar algo razonable así:

In [3]:
from sklearn.ensemble import RandomForestClassifier

rf_base = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    class_weight="balanced",
    n_jobs=-1,
    random_state=42
)


RF sin PCA sobre datos escalados (baseline PCA)

Opcional pero útil: RF + datos escalados sin PCA, para comparar con los RF+PCA.

In [4]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Entrenar RF con X_train_scaled sin PCA
rf_base.fit(X_train_scaled, y_train)
y_test_pred_rf_scaled = rf_base.predict(X_test_scaled)

acc_rf_scaled = accuracy_score(y_test, y_test_pred_rf_scaled)
prec_rf_scaled, rec_rf_scaled, f1_rf_scaled, _ = precision_recall_fscore_support(
    y_test, y_test_pred_rf_scaled, average="macro"
)

acc_rf_scaled, prec_rf_scaled, rec_rf_scaled, f1_rf_scaled


(0.7679, 0.7542461514575364, 0.7453115813389393, 0.7495835861952642)

PCA con distintos números de componentes

Ahora sí, probamos PCA con [12, 10, 8, 5, 3] componentes.


In [5]:
from sklearn.decomposition import PCA

componentes = [12, 10, 8, 5, 3]

resultados_pca = []

for n in componentes:
    print(f"\n=== PCA con {n} componentes ===")
    
    # 1. Ajustar PCA sobre X_train_scaled
    pca = PCA(n_components=n, random_state=42)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca  = pca.transform(X_test_scaled)
    
    # 2. Nuevo RandomForest (mismo config)
    rf_pca = RandomForestClassifier(
        n_estimators=100,
        max_depth=None,
        class_weight="balanced",
        n_jobs=-1,
        random_state=42
    )
    
    rf_pca.fit(X_train_pca, y_train)
    y_test_pred_pca = rf_pca.predict(X_test_pca)
    
    # 3. Métricas
    acc = accuracy_score(y_test, y_test_pred_pca)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_test, y_test_pred_pca, average="macro"
    )
    
    print(f"Accuracy test: {acc:.4f} | F1_macro: {f1:.4f}")
    
    resultados_pca.append({
        "Modelo": f"RF_PCA_{n}",
        "Componentes": n,
        "Accuracy": acc,
        "Precision_macro": prec,
        "Recall_macro": rec,
        "F1_macro": f1
    })



=== PCA con 12 componentes ===
Accuracy test: 0.6905 | F1_macro: 0.6580

=== PCA con 10 componentes ===
Accuracy test: 0.6812 | F1_macro: 0.6477

=== PCA con 8 componentes ===
Accuracy test: 0.6828 | F1_macro: 0.6481

=== PCA con 5 componentes ===
Accuracy test: 0.6490 | F1_macro: 0.6116

=== PCA con 3 componentes ===
Accuracy test: 0.6153 | F1_macro: 0.5739


Tabla final de resultados PCA + baseline

Armamos el DataFrame comparando:

RF sin PCA (datos escalados).

RF con PCA para cada N componentes.

In [6]:
import pandas as pd

# Agregar baseline sin PCA
resultados_pca.insert(0, {
    "Modelo": "RF_sin_PCA",
    "Componentes": X_train_scaled.shape[1],
    "Accuracy": acc_rf_scaled,
    "Precision_macro": prec_rf_scaled,
    "Recall_macro": rec_rf_scaled,
    "F1_macro": f1_rf_scaled
})

tabla_pca = pd.DataFrame(resultados_pca)
tabla_pca


,Modelo,Componentes,Accuracy,Precision_macro,Recall_macro,F1_macro
0,RF_sin_PCA,6700,0.76790,0.754246,0.745312,0.749584
1,RF_PCA_12,12,0.69050,0.669210,0.651335,0.658047
2,RF_PCA_10,10,0.68120,0.656672,0.642510,0.647669
3,RF_PCA_8,8,0.68280,0.657968,0.641642,0.648128
4,RF_PCA_5,5,0.64900,0.622389,0.605059,0.611601
5,RF_PCA_3,3,0.61535,0.583193,0.567783,0.573949


### Análisis de PCA (Reducción de dimensionalidad)

Debido a que, después del preprocesamiento y la codificación one-hot, la matriz de características contiene miles de variables (aproximadamente 6 700 columnas), se aplicó Análisis de Componentes Principales (PCA) con el objetivo de reducir la dimensionalidad y analizar el impacto sobre el desempeño del modelo seleccionado (Random Forest).

Previo a PCA, todas las variables de entrada se escalaron utilizando `StandardScaler`, ajustando el scaler únicamente con el conjunto de entrenamiento y transformando posteriormente el conjunto de prueba. A partir de estos datos escalados se entrenó un Random Forest en dos escenarios:

- Sin reducción de dimensionalidad (RF_sin_PCA).
- Con PCA utilizando diferentes cantidades de componentes principales: 12, 10, 8, 5 y 3 componentes.

Para cada configuración se evaluó el modelo en el conjunto de prueba mediante Accuracy, Precision macro, Recall macro y F1 macro. Los resultados obtenidos se resumen en la siguiente tabla:

*(aquí va la tabla `tabla_pca`)*

De la tabla se observa que el modelo **Random Forest sin PCA** alcanza el mejor desempeño global, con un Accuracy cercano a 0.77 y una F1 macro aproximada de 0.75. Al aplicar PCA con **12, 10 u 8 componentes**, el rendimiento disminuye de forma moderada: la F1 macro se sitúa alrededor de 0.65 y el Accuracy baja a un rango de 0.68–0.69. 

Esta pérdida de aproximadamente **0.09–0.10 puntos de F1 macro** se incrementa al reducir todavía más la dimensionalidad: con 5 componentes la F1 macro desciende a ~0.61 y con 3 componentes a ~0.57, lo que indica que, a partir de cierto punto, la compresión elimina información relevante para la clasificación.

En síntesis:

- Si el objetivo principal es **maximizar la precisión del modelo**, la mejor opción es mantener el **Random Forest sin PCA**, utilizando todas las variables originales procesadas.
- Si se busca un **compromiso entre simplicidad del modelo y rendimiento**, trabajar con PCA y alrededor de **8–12 componentes** permite reducir drásticamente la dimensionalidad de los datos manteniendo un nivel de desempeño aceptable (F1 macro ≈ 0.65).
